In [2]:
# Find substrates that could aleviate need for biotin
import cobra
import cobra.test
import pickle
import pandas as pd
from cobra.flux_analysis import gapfilling


In [2]:
df = pd.read_csv('vitamin_to_blocked_reactions.csv', index_col=0)

In [3]:
ijo = cobra.io.load_json_model('/home/sbrg-cjlloyd/Desktop/ecoli_M_models/iJO1366.json')

In [4]:
ijo.solver = 'gurobi'

In [5]:
uni = cobra.Model()
for m in ijo.metabolites.query('_c'):
    new_r = cobra.Reaction('source_' + m.id)
    uni.add_reaction(new_r)
    new_r.add_metabolites({m: 1})

In [ ]:
result_dict = {}
for i in df.index:
    model = ijo.copy()
    for met in i.split(', '):
        if '_c' not in met:
            met += '_c'
        met_obj = model.metabolites.get_by_id(met)
        biomass = model.reactions.BIOMASS_Ec_iJO1366_core_53p95M
        if met_obj in biomass.metabolites:
            biomass.add_metabolites({met_obj:0}, combine=False)
    
    for r in df.loc[i, '0'].split(', '):
        if r in model.reactions:
            model.reactions.get_by_id(r).knock_out()
        elif r[:-1] in model.reactions:
            model.reactions.get_by_id(r[:-1]).knock_out()
        elif r == 'BTS6':
            model.reactions.get_by_id('BTS5').knock_out()
        else:
            print(r)
    print(i)
    model.solver.problem.setParam('IntFeasTol', 1e-9)
    model.solver.problem.setParam('FeasibilityTol', 1e-9)
    gfill = gapfilling.GapFiller(model, uni, lower_bound=.01, exchange_reactions=True, 
                     demand_reactions=True, integer_threshold=1e-9)
    try:
        result = gfill.fill(iterations=5)   
    except:
        continue

    result_dict[i] = result
    for i, entries in enumerate(result):
        print("---- Run %d ----" % (i + 1))
        for e in entries:
            print(e.id)

thmpp
btn
---- Run 1 ----
source_kdolipid4_c
---- Run 2 ----
source_kdo2lipid4_c
---- Run 3 ----
source_lipidAds_c
---- Run 4 ----
source_lipidA_c
---- Run 5 ----
source_u23ga_c
S_transfer_to_SufBCD_mod_2:fe2
S_transfer_to_SufBCD_mod_1:2Fe1S
S_transfer_to_SufBCD_mod_1:2Fe2S_mod_2:fe2
S_transfer_to_IscU_mod_1:2Fe2S_mod_1:2Fe1S
S_transfer_to_IscU_mod_1:2Fe2S_mod_2:fe2
S_transfer_to_SufBCD_mod_1:2Fe2S_mod_1:2Fe1S
S_transfer_to_IscU_mod_2:fe2
S_transfer_to_IscU_mod_1:2Fe1S
pydx5p
cbl1, adocbl
L__LACD3
CU2R
L__LACD2
ribflv, fad, fmn, rbflvrd, fadh2, fmnh2
nad, nadp, nadh, nadph
thf, 10fthf, 5fthf, 5mthf, methf, mlthf, mththf


In [ ]:
v.